In [1]:
from requests import get
url = 'http://www.imdb.com/search/title?release_date=2017&sort=num_votes,desc&page=1'
response = get(url)
print(response.text[:500])



<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///?src=mdot">



        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>

<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle",


In [2]:
from urllib.request import urlopen as uReq

In [3]:
from bs4 import BeautifulSoup
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

In [4]:
from bs4 import BeautifulSoup as soup

In [5]:
movie_containers = html_soup.findAll('div', {"class": "lister-item mode-advanced"})
print(type(movie_containers))
print(len(movie_containers))

<class 'bs4.element.ResultSet'>
50


In [6]:
movie_containers[0]

<div class="lister-item mode-advanced">
<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt3315342"></div>
</div>
<div class="lister-item-image float-left">
<a href="/title/tt3315342/"> <img alt="Logan" class="loadlate" data-tconst="tt3315342" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BYzc5MTU4N2EtYTkyMi00NjdhLTg3NWEtMTY4OTEyMzJhZTAzXkEyXkFqcGdeQXVyNjc1NTYyMjg@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/G/01/imdb/images/nopicture/large/film-184890147._CB470041630_.png" width="67"/>
</a> </div>
<div class="lister-item-content">
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt3315342/">Logan</a>
<span class="lister-item-year text-muted unbold">(2017)</span>
</h3>
<p class="text-muted ">
<span class="certificate">R</span>
<span class="ghost">|</span>
<span class="runtime">137 min</span>
<span class="ghost">|</span>
<span class="genre">
Ac

In [7]:
movie_containers[0].a.img["alt"]

'Logan'

In [8]:
movie_containers[0].findAll("span",{"class":"certificate"})[0].text

'R'

In [9]:
movie_containers[0].findAll("span",{"class":"genre"})[0].text.strip()

'Action, Drama, Sci-Fi'

In [10]:
movie_containers[7].findAll("span",{"class":"lister-item-year text-muted unbold"})[0].text[1:5]

'I) ('

In [11]:
movie_containers[0].findAll("span",{"class":"runtime"})[0].text[0:3]

'137'

In [12]:
movie_containers[0].strong.text

'8.1'

In [14]:
movie_containers[0].findAll("span",{"class":"metascore favorable"})[0].text.strip()

'77'

In [15]:
movie_containers[0].find('span', class_ = 'metascore').text

'77        '

In [16]:
movie_containers[0].findAll("span",{"name":"nv"})[0]["data-value"]

'558781'

In [17]:
movie_containers[0].findAll("span",{"name":"nv"})[1]["data-value"]

'226,277,068'

In [18]:
movie_containers[0].findAll("span",{"name":"nv"})[1]

<span data-value="226,277,068" name="nv">$226.28M</span>

In [22]:
from requests import get
url = 'https://www.imdb.com/search/title?release_date=2017-01-01,2017-12-31&sort=boxoffice_gross_us,desc'
response = get(url)
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)
movie_containers = html_soup.findAll('div', {"class": "lister-item mode-advanced"})

filename = "movierating.csv"
f = open(filename,"w")
headers = "name,certificate,genre,year,duration\n"
f.write(headers)



for container in movie_containers:
    name = container.a.img["alt"]
    certificate = container.findAll("span",{"class":"certificate"})[0].text
    genre = container.findAll("span",{"class":"genre"})[0].text.strip()
    year = container.findAll("span",{"class":"lister-item-year text-muted unbold"})[0].text[1:5]
    duration = container.findAll("span",{"class":"runtime"})[0].text[0:3]
    IMDbscore = container.strong.text
    Metascore = container.findAll("span",{"class":"metascore favorable"})[0].text.strip()
    print(Metascore)
    votes = container.findAll("span",{"name":"nv"})[0]["data-value"]
    #gross = container.findAll("span",{"name":"nv"})

    f.write(name.replace(","," ") + "," + certificate.replace(","," ") + "," + genre.replace(",","") + "," + year.replace("I) (","2017") + "," + duration.replace(","," ")+ "\n")

f.close()

85
65
76


IndexError: list index out of range

In [1]:
import requests 
from bs4 import BeautifulSoup 
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def start_requests(url):
    r = requests.get(url)
    # Test if the url can be parsed  
    if r.status_code != 200:
        print('Failure')
    else:
        return r.content

In [3]:
# Put all movies information into a dictionary (all info of a type as list): easy for table
movie_Dict = {'Name':[],'Year':[], 'Level':[], 'Genre':[], 'Score':[], 'Director':[], 'Votes':[], 'Gross_in_Million':[]}
# Put all movies information into a List (each movie as dictionary): easy for chart
movie_List = []

# Parse the source code of the web pages and find all the information needed
def parse(text):
    # Use Beautiful Soup to parse source code
    soup = BeautifulSoup(text, 'html.parser')
    # Find all of the information of each movie inside the same tag 'div' with arttribute "class='lister-item-content'"
    content_List = soup.find_all('div', class_ = 'lister-item-content')
    for movie in content_List:
        # According to the html & Find detailed information inside each movie
        movies_Name = movie.find_all('a')[0].text
        # Since some parts of movie info are not available, Exceptions should be applied here.
        try:
            movies_Year = movie.find_all('span')[1].text
            movies_ReleaseYear = re.findall(r'\d{4}', movies_Year)[0]
        except:
            movies_ReleaseYear = 'No Year Info'
        try:
            movies_R_Level = movie.find_all('span', class_ = 'certificate')[0].text.strip()
        except:
            movies_R_Level = 'Not Rated'
        try:
            movies_Genre = movie.find_all('span', class_ = 'genre')[0].text.strip().replace(',','')
        except:
            movies_Genre = 'Not Classified'
        try:
            movies_Score = float(movie.find_all('div', class_ = 'inline-block ratings-imdb-rating')[0]['data-value'])
        except:
            movies_Score = 0
        try:
            movies_Director = movie.find_all('p')[2].find_all('a')[0].text
        except:
            movies_Director = 'No Director Info'
        try: 
            movies_Votes = int(movie.find_all('p')[3].find_all('span')[1]['data-value'])
        except:
            movies_Votes = 0
        try: 
            movies_Gross = int(movie.find_all('p')[3].find_all('span')[4]['data-value'].replace(',',''))/1000000
        except:
            movies_Gross = 0
            
        movie_Dict['Name'].append(movies_Name)
        movie_Dict['Year'].append(movies_ReleaseYear)
        movie_Dict['Level'].append(movies_R_Level)
        movie_Dict['Genre'].append(movies_Genre)
        movie_Dict['Score'].append(movies_Score)
        movie_Dict['Director'].append(movies_Director)
        movie_Dict['Votes'].append(movies_Votes)
        movie_Dict['Gross_in_Million'].append(movies_Gross)
        
        movie_List.append({'Name': movies_Name, 'Year': movies_ReleaseYear, 'Level':movies_R_Level, 
                            'Genre': movies_Genre, 'Score': movies_Score, 'Director':movies_Director,
                            'Votes': movies_Votes, 'Gross_in_Million':movies_Gross})

# Main function: call the other functions
def main():
    numbers = int(input("Please enter a number of page(s) you want to crawl (it should be an integer between 1 to 8048):"))
    # Require an integer input between 1 to 8048
    if numbers > 0 and numbers < 8049:
        # For Loop: Make the crawler be able to crawl multiple pages (url)
        for i in range(numbers):
            x = 1 + (i * 50)
            url = f'https://www.imdb.com/search/title?title_type=feature&start={x}&ref_=adv_nxt'
            text = start_requests(url)
            parse(text)
    else:
        raise ValueError('Number Invalid. Please enter again, thank you!')

if __name__ == '__main__':
    main()

# print(movie_Dict)
# len(movie_Dict['Year'])
# print(movie_List)

Please enter a number of page(s) you want to crawl (it should be an integer between 1 to 8048):1000


In [4]:
Dataframe = pd.DataFrame(movie_Dict)   #organize data we got into a csv file to make it clear
Dataframe[[ 'Name','Year','Level', 'Genre','Score','Director','Votes','Gross_in_Million']].to_csv('movies.csv') #export data to csv

In [5]:
movies_table = pd.read_csv("movies.csv") #read csv file in python
Dataframe = movies_table[['Name','Year','Level', 'Genre','Score','Director','Votes','Gross_in_Million']]#create table
Dataframe.head(150) #use 50 movies as example

,Name,Year,Level,Genre,Score,Director,Votes,Gross_in_Million
0,Aladdin,2019,PG,Adventure Comedy Family,7.4,Guy Ritchie,45488,185.537718
1,Godzilla: King of the Monsters,2019,PG-13,Action Adventure Fantasy,6.9,Michael Dougherty,22450,47.776293
2,Rocketman,2019,R,Biography Drama Music,7.7,Dexter Fletcher,14552,25.725722
3,Avengers: Endgame,2019,PG-13,Action Adventure Sci-Fi,8.8,Anthony Russo,430398,815.726275
4,John Wick: Chapter 3 - Parabellum,2019,R,Action Crime Thriller,8.0,Chad Stahelski,70804,125.738271
5,Booksmart,2019,R,Comedy,7.5,Olivia Wilde,13689,14.339576
6,Once Upon a Time ... in Hollywood,2019,Not Rated,Comedy Drama,9.8,Quentin Tarantino,5050,0.000000
7,Terminator: Dark Fate,2019,Not Rated,Action Adventure Sci-Fi,0.0,Tim Miller,0,0.000000
8,The Perfection,2018,TV-MA,Drama Horror Thriller,6.2,Richard Shepard,10569,0.000000
9,Brightburn,2019,R,Horror Sci-Fi,6.5,David Yarovesky,10726,14.353235
